# Numpy-based Neural Network engine from scratch

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/m12sl/dl-hse-2020/blob/master/01-introduction/numpy%20neural%20networks%20from%20scratch.ipynb)



В этой тетрадке мы напишем собственный простенький фреймворк для создания и обучения нейронной сети.

**Цели тетрадки**

1. Понять, что лежит в основе DL фреймворков
2. Разобраться с бекпропом на примере простых слоев
3. Встретить характерные для векторизованных вычислений ошибки
4. Вспомнить особенности операций с плавающей точкой
5. Попробовать тестировать код сразу


**План работы**

0. Обсудить векторизацию вычислений и договориться о формате входных-выходных данных
1. Рассмотреть как высокоуровнево устроен движок для сеток
2. Реализовать несколько базовых слоев
3. Натренировать простую сеть на задаче MNIST

In [ ]:
%matplotlib inline
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np

from tqdm.auto import tqdm

# Векторизация вычислений (batchification)

<img src="https://github.com/m12sl/dl-hse-2020/raw/master/01-introduction/img/graph_vector.png" crossorigin="anonymous"/>

В современных компьютерах выгодно объединять данные к которым будет применена одна и та же операция в вектора и батчи. Для вычисления длин нескольких векторов их, как правило, выгодно склеить в массив (numpy.ndarray) и применить векторизованную операцию (все numpy-операции работают с многомерными массивами).


На практике во всех фреймворках для DL (да и для ML) работают с данными упакованными в батчи: 
многомерные массивы с размерами `[batch_size, channels, *spatial_dimensions]` -- такие массивы называются тензорами. 

Подразумевается, что примеры в батче независимы друг от друга и никак не влияют друг на друга (за исключением некоторых нормализаций и специальных трюков).


<img src="https://github.com/m12sl/dl-hse-2020/raw/master/01-introduction/img/graph_batched.png" crossorigin="anonymous"/>


Предлагается реализовать простой движок для обучения простых нейронных сетей (MLP, Multi Layer Perceptron).

Класс для обходов вычислительного графа предлагается взять таким:

In [ ]:
class SomeNetwork:
    def __init__(self, layers):
        self.layers = layers
    
    def forward(self, input):
        x = input
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self, grads):
        for layer in self.layers[::-1]:
            grads = layer.backward(grads)
        return grads

Слои -- это объекты с двумя методами (`forward`, `backward`) и конструктором (`__init__`).

**Для простоты предлагается вычисление градиентов по весам и их применение объединить с backward-проходом.**

Операция `Wx + b` формально имеет три входа: входной тензор $x$ и веса: $W$ и $b$. Мы разместим веса в слое, так что слой на прямом проходе будет иметь только один вход `x`, а на обратно выдавать градиенты $\frac{\partial L}{\partial x}$.


**Посмотрите внимательно на прототип слоя, вам потребуется реализовать Linear и ReLU на основе него:**

In [ ]:
# Это на самом деле identity
class Layer:
    def forward(self, input):
        # store something if needed
        return input
    
    def backward(self, grads):
        # apply grads here if any
        return grads

In [ ]:
# Сеть которая не делает ничего:
network = SomeNetwork([Layer(), Layer()])

x = np.random.random(size=(3, 5))
output = network.forward(x)
assert output.shape == (3, 5)

y = np.ones((3, 5))
grads = network.backward(y)
assert grads.shape == (3, 5)

В качестве сети возьмем самую простую: Multilayer Perceptron (MLP).

<img src="https://github.com/m12sl/dl-hse-2020/raw/master/01-introduction/img/graph_mlp.png" crossorigin="anonymous"/>


Нам потребуется реализовать следующие слои:

```
Linear
Relu
```

Целевой задачей возьмем многоклассовую классификацию, мы хотим, чтобы наша сеть для каждого входного примера предсказывала распределение вероятностей $q_i$.

Функция ошибки должна отражать близость распределения $q$ к известным правильным ответам. В этом нам поможет кросс-энтропийная функция ошибки (перекрестная энтропия):
$$
H(p, q) = - \sum \limits_{i} p_i \log q_i 
$$

Эта формула описывает близость между двумя распределениями вероятностей:
 - $i$ --- номер класса,
 - $p_i$ --- правильная вероятность класса (единица, еcли правильная метка i, ноль, если нет)
 - $q_i$ --- предсказанная вероятность (выход сети).

## Linear Layer

Синонимы: Linear, Dense, FullyConnected, Projection:

Математическое действие над вектором признаков:
$$\mathbf{y} = W \mathbf{x} + \mathbf{b}$$

Пусть 
$\mathbf{x} \in \mathbb{R}^n$
$\mathbf{y}\in \mathbb{R}^m$

**Какую размерность должен иметь $W$ и $b$?**

Давайте теперь реализуем векторизованный вариант слоя: `y = f(x, W, b)`

```python
x.shape == (batch_size, in_channels)
y.shape == (batch_size, out_channels)
```

**0.5 балла** Напишите код для вычисления прямого прохода и обратного в векторизованной форме.
Шаг градиентного спуска предлагается совместить с обратным проходом.

Веса $W, b$ считаем принадлежащими слою, инициализируем при создании, во время обратного прохода возвращаем градиенты только за входной тензор.

**NB: нас не интересует строгое соответствие математической нотации, она годится для работы с векторами. Мы же работаем с данными собранными в батчи.**

Для простоты движка важно, чтобы операции получали и возвращали данные в правильном формате. 
В данном случае -- это numpy.ndarray с размерностями `[bs, dim]`.

In [ ]:
# Реализуйте линейный слой
# y = Wx + b

class Linear(Layer):
    def __init__(self, in_channels, out_channels, learning_rate=0.1):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.W = np.random.randn(in_channels, out_channels) * 0.01
        self.b = np.zeros(out_channels)
        self.learning_rate = learning_rate
        # for debug purpose
        self.grad_W = None
        self.grad_b = None
        self.grad = None
    
    def forward(self, x):
        <напишите свой код>
        return y
    
    def backward(self, grad):
        <напишите вычисление градиентов>
        
        self.W = self.W - grad_W * self.learning_rate
        self.b = self.b - grad_b * self.learning_rate
        
        self.grad_W = grad_W
        self.grad_b = grad_b
        self.grad = grad
        return grad

    
# тесты для самопроверки
linear = Linear(5, 7)

x = np.random.random(size=(3, 5))
assert linear.forward(x).shape == (3, 7), "Forward pass shape mistmatch"

y = np.random.random(size=(3, 7))
grad_x = linear.backward(y)
assert grad_x.shape == (3, 5)
assert linear.grad_W.shape == linear.W.shape
assert linear.grad_b.shape == linear.b.shape

## ReLU Layer

Это простая поэлементная операция, записывается как $f(x) = max(x, 0)$.

**NB: активационные слои не меняют размер тензора**

**0.3 балла**

In [ ]:
class ReLU(Layer):
    def forward(self, input):
        <напишите код>
    
    def backward(self, grad):
        <напишите код>
    
relu = ReLU()
x = np.random.normal(size=(3, 5))
assert relu.forward(x).shape == (3, 5)
assert np.all(relu.forward(x) >= 0.0)

y = np.random.normal(size=(3, 5))
grads = relu.backward(y)
assert grads.shape == (3, 5) 

approx = (relu.forward(x + 1e-5) - relu.forward(x)) / 1e-5

relu.forward(x)
assert np.allclose(relu.backward(np.ones_like(x)), approx)

Для подсчета ошибки нам необходимы предсказания сети и правильные ответы.

Для простоты подсчет ошибки и градиентов вынесены в отдельные функции вместе с вычислением вероятностей (softmax).

Каждая из функций ожидает на вход логиты -- тензор с размерами `[bs, num_classes]`.

`softmax_crossentropy_with_logits` возвращает одно число -- величину ошибки.

`grad_softmax_crossentropy_with_logits` возвращает тензор размером `[bs, num_classes]` с градиентами по логитам.


In [ ]:
def softmax_crossentropy_with_logits(logits, reference_answers):
    """Compute crossentropy from logits[batch, n_classes] and ids of correct answers"""
    logits_for_answers = logits[np.arange(len(logits)),reference_answers]
    
    xentropy = - logits_for_answers + np.log(np.sum(np.exp(logits),axis=-1))
    
    return xentropy.mean()

def grad_softmax_crossentropy_with_logits(logits, reference_answers):
    """Compute crossentropy gradient from logits[batch,n_classes] and ids of correct answers"""
    ones_for_answers = np.zeros_like(logits)
    ones_for_answers[np.arange(len(logits)),reference_answers] = 1
    
    softmax = np.exp(logits) / np.exp(logits).sum(axis=-1,keepdims=True)
    
    return (- ones_for_answers + softmax) / logits.shape[0]

## Data Handling
Для работы с данными во всех фреймворках есть Dataset/Dataloader классы.

Мы будем разбирать pytorch Data API отдельно, а пока предлагается воспользоваться готовым кодом:

In [ ]:
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST, M

# в FashionMNIST уже есть код, который грузит данные и читает их в PIL.Image, мы конвертируем их в вектора.
# Непосредственно с картинками мы будем работать позже
def transform(pil_image):
    return np.asarray(pil_image).astype(np.float32).reshape(-1) / 255.0


def numpy_collate_fn(batch):
    x, y = list(zip(*batch))
    return np.array(x).astype(np.float32), np.array(y).astype(np.int32)


# dataset умеет выдавать отдельные семплы с метками
train_dataset = MNIST("./tmp2", train=True, download=True, transform=transform)
test_dataset = MNIST("./tmp2", train=False, download=True, transform=transform)
# dataloader выдает целые батчи и поддерживает итерирование
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=32, collate_fn=numpy_collate_fn)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=32, collate_fn=numpy_collate_fn)

In [ ]:
# dataset возвращает отдельные семплы
print(train_dataset[11][0].shape, train_dataset[11][1])

# dataloader целые батчи
for x, y in train_loader:
    print(x.dtype, y.dtype)
    print(x.shape, y.shape)
    break

plt.figure(figsize=[6,6])
for i in range(4):
    plt.subplot(2,2,i+1)
    plt.title(f"Class: {train_dataset[i][1]}")
    plt.imshow(train_dataset[i][0].reshape(28, 28),cmap='gray');

## Train Loop

Теперь все готово чтобы сделать сеть и проучить ее.
Если все реализовано правильно, можно ожидать точности на валидации ~0.86+.

Кросс-энтропийный лосс не слишком показательная величина, но если он должен оказаться ~0.4 на валидации.

In [ ]:
network = SomeNetwork([
    Linear(784, 40),
    ReLU(), 
    Linear(40, 40),
    ReLU(),
    Linear(40, 10),
])

epochs = 10
for epoch in range(epochs):
    for x, y in tqdm(train_loader):
        logits = network.forward(x)
        loss = softmax_crossentropy_with_logits(logits, y)
        acc = np.mean(np.argmax(logits, axis=-1) == y)
        grads = grad_softmax_crossentropy_with_logits(logits, y)
        network.backward(grads)
        
    logs = defaultdict(list)
    for x, y in tqdm(test_loader):
        logits = network.forward(x)
        loss = softmax_crossentropy_with_logits(logits, y)
        acc = np.mean(np.argmax(logits, axis=-1) == y)
        logs['loss'].append(loss)
        logs['acc'].append(acc)
        
    for k, v in logs.items():
        print(k, np.mean(v))

**0.1 балл**. Постройте графики обучения: метрики от количества шагов.


**0.1 балл**. Выведите пример картинок, предсказаний и правильных ответов.